In [16]:
## importing python libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## importing csv files

crystal_data = pd.read_csv('crystal_dataset - sheet.csv')
substrate_data = pd.read_csv('substrate_data - Sheet1.csv')
edge_substrate_data = substrate_data['a(A)']
edge_substrate_data_lst = edge_substrate_data.values.tolist()

## changing to float values

edge_substrate_data_lst_float = []
for k in edge_substrate_data_lst:
  x = round(float(k), 3)
  edge_substrate_data_lst_float.append(x)
    
edge_crystal_data = crystal_data['a(A)']
edge_crystal_data_lst = edge_crystal_data.values.tolist()

## changing to float values

edge_crystal_data_lst_float = []
for l in edge_crystal_data_lst:
  y = round(float(l), 3)
  edge_crystal_data_lst_float.append(y)

## calculating mismatch

num = 0
for j in edge_crystal_data_lst_float:
  idx = edge_crystal_data_lst_float.index(j)
  sample = crystal_data['formula'][idx]

 # creating variables

  globals()["length_difference" + str(num)] = []
  globals()["sqrtlength_difference" + str(num)] = []
  globals()["length_difference_percent" + str(num)] = []
  globals()["sqrtlength_difference_percent" + str(num)] = []
  globals()["x" + str(num)] = []
    
  for i in edge_substrate_data_lst_float:
    diff_100 = j - i
    diff_110 = j - 1.414*(i)
    diff_100_rounded = round(diff_100, 3)
    diff_110_rounded = round(diff_110, 3)
    percent_100 = ((diff_100/i)*100)
    percent_100_rounded = round(percent_100, 3)
    percent_110 = (diff_110 /(1.414 * i))*100
    percent_110_rounded = round(percent_110, 3)
    
    globals()["length_difference" + str(num)].append(diff_100_rounded)
    globals()["sqrtlength_difference" + str(num)].append((diff_110_rounded))
    globals()["length_difference_percent" + str(num)].append(percent_100_rounded)
    globals()["sqrtlength_difference_percent" + str(num)].append(percent_110_rounded)
    globals()["x" + str(num)].append(i)

  
 # creating_dataframe

  globals()["length_difference_percent"+str(sample)] = pd.DataFrame(globals()["length_difference_percent" + str(num)],columns =['lattice_mismatch_percent_100'])
  globals()["stress_type100" + str(sample)]  = []
  globals()["sqrtlength_difference_percent"+str(sample)] = pd.DataFrame(globals()["sqrtlength_difference_percent" + str(num)],columns =['lattice_mismatch_percent_110'])
  globals()["stress_type110" + str(sample)]  = []
    
  for i in globals()["length_difference_percent" + str(num)]:
        if i > 0:
            globals()["stress_type100" + str(sample)].append('compressive_stress')
        elif i < 0:
            globals()["stress_type100" + str(sample)].append('tensile_stress')
        else:
            globals()["stress_type100" + str(sample)].append('no _stress')
            
  for p in globals()["sqrtlength_difference_percent" + str(num)]:
        if p > 0:
            globals()["stress_type110" + str(sample)].append('compressive_stress')
        elif p < 0:
            globals()["stress_type110" + str(sample)].append('tensile_stress')
        else:
            globals()["stress_type110" + str(sample)].append('no _stress')
  
  globals()["stress_type100"+str(sample)] = pd.DataFrame(globals()["stress_type100"+str(sample)],columns =['stress_type'])
  globals()["stress_type110"+str(sample)] = pd.DataFrame(globals()["stress_type110"+str(sample)],columns =['stress_type'])
  globals()["mismatch_data"+str(sample)] = pd.concat([substrate_data['Substrate Name'],globals()["length_difference_percent"+str(sample)],globals()["stress_type100"+str(sample)],globals()["sqrtlength_difference_percent"+str(sample)],globals()["stress_type110"+str(sample)]], axis = 1)
  index = globals()["mismatch_data"+str(sample)].index
  index.name = str(sample)
  globals()["df"+str(sample)] = pd.DataFrame(globals()["mismatch_data"+str(sample)])
  globals()["df"+str(sample)].to_csv(str(sample)+'.csv',index = False)
  
  ## scatter plotting

  plt.style.use('seaborn-darkgrid')
  plt.rcParams["figure.figsize"] = (11, 7.5)
  plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
  globals()["xfit" + str(num)] = np.array(globals()["x" + str(num)]) 
  globals()["length_difference_percentfit100" + str(num)] = np.array( globals()["length_difference_percent" + str(num)])
  globals()["length_difference_percentfit110" + str(num)] = np.array( globals()["sqrtlength_difference_percent" + str(num)])  
  xfit = np.arange(3,6,0.01)
  z = np.polyfit(globals()["xfit" + str(num)],globals()["length_difference_percentfit100" + str(num)], 2)
  globals()["yfit100" + str(num)] = []

  for i in xfit :
    b = z[0]*i*i + z[1]*i + z[2]
    globals()["yfit100" + str(num)].append(b)
    
  plt.plot(xfit,globals()["yfit100" + str(num)],linestyle='--')

  e = np.polyfit(globals()["xfit" + str(num)],globals()["length_difference_percentfit110" + str(num)], 2)
    
  globals()["yfit110" + str(num)] = []

  for i in xfit :
    b = e[0]*i*i + e[1]*i + e[2]
    globals()["yfit110" + str(num)].append(b)
    
  plt.plot(xfit,globals()["yfit110" + str(num)],linestyle='--')

  globals()["fig100" + str(num)] = plt.scatter(globals()["x" + str(num)], globals()["length_difference_percent" + str(num)], label = '100')
  globals()["fig110" + str(num)] = plt.scatter(globals()["x" + str(num)], globals()["sqrtlength_difference_percent" + str(num)], label = '110')

  plt.title('lattice_mismatch_plot_of_' + str(sample),fontsize=30)
  plt.yticks(np.arange(-40, 40, 5))
  plt.xticks(np.arange(3.50,5.75,0.25))
  plt.xlabel('substrate_edge_length',fontsize=20)
  plt.ylabel('lattice_mismatch',fontsize=20)
  plt.axis([3.50,5.75,-40,40])
  plt.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1,fontsize=15)
  x = np.arange(3,6.5,0.1)  

  # plotting the limits
  a1 = 10
  a2 = -10
  plt.fill_between(x,a1,a2,color='green', alpha=0.3)
    
  # marking the points  

  globals()["marking_points_100" + str(num)] = []
  globals()["marking_points_110" + str(num)] = []  
  for z,k in zip(globals()["length_difference_percent" + str(num)],globals()["sqrtlength_difference_percent" + str(num)]):
    globals()["marking_points_100" + str(num)].append(z)
    globals()["marking_points_110" + str(num)].append(k)
    
  num2 = 1
  for c,j in zip(globals()["marking_points_100" + str(num)],globals()["marking_points_110" + str(num)]):
    ind = globals()["length_difference_percent" + str(num)].index(c)
    temp = globals()["x" + str(num)][ind]
    mark = substrate_data['Substrate Name'][ind]
    globals()["point" + str(num2)] = mark
    plt.annotate(str(mark),(temp,c),xytext=(0,10), textcoords='offset points', horizontalalignment='left', verticalalignment='bottom',fontsize=12)
    ind2 = globals()["sqrtlength_difference_percent" + str(num)].index(j)
    temp2 = globals()["x" + str(num)][ind2]
    mark2 = substrate_data['Substrate Name'][ind2]
    globals()["point" + str(num2)] = mark2
    plt.annotate(str(mark2),(temp2,j),xytext=(0,10), textcoords='offset points', horizontalalignment='left', verticalalignment='top',fontsize=12)
  num2 += 1
    
  plt.savefig(str(num)+'.jpeg')
  plt.clf()
  num += 1

<Figure size 792x540 with 0 Axes>